# Projeto 1 - Ciência dos Dados

Nome: Duilio

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'dados.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo dados.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,"rt @dudacoutin_: vontade de comer brigadeiro, ...",1
1,qm q vai faze um brigadeiro branco p mim to co...,1
2,@vallnicolau tá agora eu to com vontade de com...,1
3,me deu uma vontade de comer brigadeiro com mor...,1
4,tava com vontade de comer abacaxi e brigadeiro...,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,meu deus ea vontade d um brigadeiro pra hj
1,"to morrendo de vontade de um brigadeiro, mas s..."
2,vontade de comer brigadeiro quase queimado
3,uma vontade imensa de comer brigadeiro tô nem ...
4,a vontade de comer brigadeiro tá grande


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Estamos trabalhando com uma empresa de venda de brigadeiros, e para isso queremos ver a quantidade de possiveis clientes para essa loja, então filtramos as pessoas que dizem estar com vontade de brigadeiro, e não produtos com, ou que eles dizem que vão fazer seu próprio brigadeiro, alem de outros tweets irrelevantes para a loja

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**